In [18]:
#Importing Libraries
import dlib
import cv2
import numpy as np
import glob
import random
import math
import itertools
from sklearn.svm import SVC
import pickle
import os
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
import time
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
emotions = ["happy","sadness","surprise","fear","disgust","anger","contempt"]

#Fetching the landmarks dat which contains the pre defined model
detector = dlib.get_frontal_face_detector()
model = dlib.shape_predictor("C:\\Users\\Neha\\Desktop\\AIProject\\shape_predictor_68_face_landmarks.dat")
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Set the classifier as a support vector machines with linear kernel
n_estimators = 10
clf_new = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear', probability=True), max_samples=1.0 / n_estimators, n_estimators=n_estimators))
clf = SVC(C=0.01, kernel='linear', probability=True)

In [20]:
def get_landmarks(image):
    detections = detector(image, 1)
    # For all detected face instances individually
    for k, d in enumerate(detections):
        # Get facial landmarks with prediction model
        shape = model(image, d)
        xpoint = []
        ypoint = []
        for i in range(0, 68):
            xpoint.append(float(shape.part(i).x))
            ypoint.append(float(shape.part(i).y))

        # Center points of both axis
        xcenter = np.mean(xpoint)
        ycenter = np.mean(ypoint)
        # Calculate distance between particular points and center point
        xdistcent = [(x-xcenter) for x in xpoint]
        ydistcent = [(y-ycenter) for y in ypoint]

        # Prevent divided by 0 value
        if xpoint[11] == xpoint[14]:
            angle_nose = 0
        else:
            # Point 14 is the tip of the nose, point 11 is the top of the nose brigde
            angle_nose = int(math.atan((ypoint[11]-ypoint[14])/(xpoint[11]-xpoint[14]))*180/math.pi)

        # Get offset by finding how the nose brigde should be rotated to become perpendicular to the horizontal plane
        if angle_nose < 0:
            angle_nose += 90
        else:
            angle_nose -= 90

        landmarks = []
        for cx, cy, x, y in zip(xdistcent, ydistcent, xpoint, ypoint):
            # Add the coordinates relative to the centre of gravity
            landmarks.append(cx)
            landmarks.append(cy)

            # Get the euclidean distance between each point and the centre point (the vector length)
            meanar = np.asarray((ycenter,xcenter))
            centpar = np.asarray((y,x))
            dist = np.linalg.norm(centpar-meanar)

            # Get the angle the vector describes relative to the image, corrected for the offset that the nosebrigde
            # has when the face is not perfectly horizontal
            if x == xcenter:
                angle_relative = 0
            else:
                angle_relative = (math.atan(float(y-ycenter)/(x-xcenter))*180/math.pi) - angle_nose
            landmarks.append(dist)
            landmarks.append(angle_relative)

    if len(detections) < 1:
        # In case no case selected, print "error" values
        landmarks = "error"
    return landmarks


In [21]:
import matplotlib.pyplot as plt
def get_files(emotion):
    images = glob.glob("C:\\Users\\Neha\\Desktop\\AIProject\\CK_DATASET\\CK_DATASET\\CK+48\\%s\\*" %emotion)
    random.shuffle(images)
    #get 80% of image files to be trained
    training_set = images[:int(len(images)*0.8)]
    #get 20% of image files to be tested
    testing_set = images[-int(len(images)*0.2):]   
    return training_set, testing_set

In [22]:
def make_sets():
    training_data = []
    training_label = []
    testing_data = []
    testing_label = []
    for emotion in emotions:
        training_set, testing_set = get_files(emotion)
        print("length of train data set:", len(training_set))
        print('length of test data set:',testing_set)
        #add data to training and testing dataset, and generate labels 0-4
        for item in training_set:
            #read image
            img = cv2.imread(item)
            #convert to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            clahe_img = clahe.apply(gray_img)
            landmarks_vec = get_landmarks(clahe_img)

            if landmarks_vec == "error":
                pass
            else:
                training_data.append(landmarks_vec)
                training_label.append(emotions.index(emotion))

        for item in testing_set:
            img = cv2.imread(item)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            clahe_img = clahe.apply(gray_img)
            landmarks_vec = get_landmarks(clahe_img)
            if landmarks_vec == "error":
                pass
            else:
                testing_data.append(landmarks_vec)
                testing_label.append(emotions.index(emotion))

    return training_data, training_label, testing_data, testing_label

# SVM

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10,0.01, 0.001, 0.0001],
              'kernel': ['linear']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
X_train, y_train, X_test, y_test = make_sets()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
  
# fitting the model for grid search
grid.fit(X_train_scaled, y_train)

length: 165
length: 67
length: 199
length: 60
length: 141
length: 108
length: 43
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.815, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.898, total=   0.0s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.924, total=   0.1s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.885, total=   0.1s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.878, total=   0.1s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.815, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.892, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.911, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.891, total=   0.1s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.878, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.815, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.9s finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 0.01, 0.001, 0.0001],
                         'kernel': ['linear']},
             verbose=3)

In [24]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.01, 'kernel': 'linear'}
SVC(C=0.01, kernel='linear')


In [25]:
grid_predictions = grid.predict(X_test_scaled)
# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.67      0.75      0.71        16
           2       1.00      0.98      0.99        49
           3       1.00      0.87      0.93        15
           4       0.94      0.97      0.96        35
           5       0.80      0.89      0.84        27
           6       0.62      0.50      0.56        10

    accuracy                           0.91       193
   macro avg       0.86      0.85      0.85       193
weighted avg       0.91      0.91      0.91       193



# KNN

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
X_train_knn, y_train_knn, X_test_knn, y_test_knn = make_sets()
print(len(X_train_knn))
print(len(y_train_knn))
X_train_knn_arr = np.array(X_train_knn)
Y_train_knn_arr = np.array(y_train_knn)
X_test_knn_arr = np.array(X_test_knn)
Y_test_knn_arr = np.array(y_test_knn)
X_train_knn_arr_scaled = scaler.fit_transform(X_train_knn_arr)
X_test_knn_arr_scaled = scaler.transform(X_test_knn_arr)

length: 165
length: 67
length: 199
length: 60
length: 141
length: 108
length: 43
783
783


In [27]:
print(X_train_knn_arr_scaled.shape)
print(Y_train_knn_arr.shape)

(783, 272)
(783,)


In [28]:
clf_knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 30) }
knn_gridcv = GridSearchCV(clf_knn, param_grid ,cv=10, refit = True,n_jobs= -1)
knn_gridcv.fit(X_train_knn_arr_scaled,Y_train_knn_arr)
knn_gridcv.best_params_

{'n_neighbors': 1}

In [29]:
scores = knn_gridcv.cv_results_['mean_test_score']
scores

array([0.92594937, 0.84543979, 0.82370983, 0.80197988, 0.77387212,
       0.77510549, 0.75725414, 0.75082765, 0.74952937, 0.7507952 ,
       0.74055501, 0.7316618 , 0.73162934, 0.73161311, 0.72010711,
       0.71752678, 0.71496267, 0.71366439, 0.71366439, 0.71239857,
       0.70217462, 0.70730282, 0.70473872, 0.70342421, 0.7009088 ,
       0.6958455 , 0.69712756, 0.69967543, 0.70350536])

In [30]:
grid_predictin_random= knn_gridcv.predict(X_test_knn_arr_scaled)
print(classification_report(Y_test_knn_arr,grid_predictin_random))

              precision    recall  f1-score   support

           0       0.95      0.88      0.91        41
           1       0.93      0.88      0.90        16
           2       1.00      0.92      0.96        49
           3       0.94      1.00      0.97        15
           4       0.85      0.80      0.82        35
           5       0.71      0.93      0.81        27
           6       0.91      1.00      0.95        10

    accuracy                           0.90       193
   macro avg       0.90      0.91      0.90       193
weighted avg       0.91      0.90      0.90       193

